In [1]:
import mysql.connector
import pandas as pd
import logging

In [3]:
# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
# Define the database connection parameters
db_config = {
    'user': 'daniel896733o',
    'password': '4#LgJPi5SfnJEb#rmDLo!#g8X7m64zaR',
    'host': 'artemis-db-100-02-rep-01.c4hc4rp7skgd.eu-west-1.rds.amazonaws.com',
    'database': 'korapay_core_engine'
}

In [5]:
%%time
# Connect to the database
connection = mysql.connector.connect(**db_config)
# Define the query
query = """
SELECT
    a.name AS "merchant",
    c.name AS "customer_name",
    c.email AS "customer_email",
    c.phone AS "customer_phone_number",
    p.reference AS "payment_reference",
    ps.reference AS "transaction_reference",
    ps.merchant_bears_cost,
    p.created_at AS "payment_created_at",
    p.completed_at AS "payment_completed_at",
    p.updated_at AS "payment_updated_at",
    p.for_disbursement_wallet,
    p.allowed_payment_methods,
    a.created_at AS "account_created_date",
    a.risk_level AS "account_risk_level",
    ps.payment_source_type,
    ps.processor AS "processor",
    ps.status AS "payment_status",
    ps.channel AS "payment_channel",
    ps.amount_collected,
    ps.payment_reversals_type,
    ps.currency AS "currency",
    t.mobile_number AS "mobile_number_for_mobile_money",
    t.network_provider AS "mobile_network_provider",
    t.processor_response AS "processor_response",
    ps.message AS "kora_translated_processor_response",
    ps.amount AS "amount",
    t.auth_model,
    ps.created_at AS "tran_date",
    0 AS "is_fraud"
FROM korapay_mobile_money.transaction_nsano_mobile_money t
INNER JOIN payment_sources ps ON ps.processor_reference = t.reference
INNER JOIN payments p ON p.id = ps.payment_id
INNER JOIN accounts a ON a.id = p.account_id
INNER JOIN customers c ON c.id = p.customer_id
WHERE DATE(t.created_at) BETWEEN '2025-01-21' AND '2025-02-28'
AND processor_response NOT IN ('TARGET_AUTHORIZATION_ERROR', 'AUTHORIZATION_SENDER_ACCOUNT_NOT_ACTIVE', 'ACCOUNTHOLDER_WITH_FRI_NOT_FOUND', 'RESOURCE_NOT_FOUND')
UNION
SELECT
    a.name AS "merchant",
    c.name AS "customer_name",
    c.email AS "customer_email",
    c.phone AS "customer_phone_number",
    p.reference AS "payment_reference",
    ps.reference AS "transaction_reference",
    ps.merchant_bears_cost,
    p.created_at AS "payment_create_date",
    p.completed_at AS "payment_complete_date",
    p.updated_at AS "payment_updated_at",
    p.for_disbursement_wallet,
    p.allowed_payment_methods,
    a.created_at AS "account_create_date",
    a.risk_level AS "account_risk_level",
    ps.payment_source_type,
    ps.processor AS "processor",
    ps.status AS "payment_status",
    ps.channel AS "payment_channel",
    ps.amount_collected,
    ps.payment_reversals_type,
    ps.currency AS "currency",
    t.mobile_number AS "mobile_number_for_mobile_money",
    t.network_provider AS "mobile_network_provider",
    t.processor_response AS "processor_response",
    ps.message AS "kora_translated_processor_response",
    ps.amount AS "amount",
    t.auth_model,
    ps.created_at AS "tran_date",
    1 AS "is_fraud"
FROM korapay_mobile_money.transaction_nsano_mobile_money t
INNER JOIN payment_sources ps ON ps.processor_reference = t.reference
INNER JOIN payments p ON p.id = ps.payment_id
INNER JOIN accounts a ON a.id = p.account_id
INNER JOIN customers c ON c.id = p.customer_id
WHERE DATE(t.created_at) BETWEEN '2025-01-21' AND '2025-02-28'
AND processor_response IN ('TARGET_AUTHORIZATION_ERROR', 'AUTHORIZATION_SENDER_ACCOUNT_NOT_ACTIVE', 'ACCOUNTHOLDER_WITH_FRI_NOT_FOUND', 'RESOURCE_NOT_FOUND')
ORDER BY "merchant", "mobile_number_for_mobile_money";
"""

2025-03-26 09:41:31,747 - INFO - package: mysql.connector.plugins
2025-03-26 09:41:31,748 - INFO - plugin_name: mysql_native_password
2025-03-26 09:41:31,754 - INFO - AUTHENTICATION_PLUGIN_CLASS: MySQLNativePasswordAuthPlugin
2025-03-26 09:41:31,981 - INFO - package: mysql.connector.plugins
2025-03-26 09:41:31,983 - INFO - plugin_name: caching_sha2_password
2025-03-26 09:41:31,985 - INFO - AUTHENTICATION_PLUGIN_CLASS: MySQLCachingSHA2PasswordAuthPlugin


CPU times: user 119 ms, sys: 19.1 ms, total: 138 ms
Wall time: 1.59 s


In [7]:
%%time
# Define chunk size
chunk_size = 10000  # Adjust based on memory capacity
# Initialize an empty list to hold each chunk DataFrame
df_list = []
try:
    for chunk in pd.read_sql(query, connection, chunksize=chunk_size):
        # Process each chunk
        logging.info(f"Processing chunk with {len(chunk)} rows")
        # Add chunk to the list of dataframes
        df_list.append(chunk)
        # Log memory usage info
        logging.info(f"Memory usage after chunk: {chunk.memory_usage(deep=True).sum() / (1024 ** 2):.2f} MB")
except Exception as e:
    logging.error(f"An error occurred while reading data in chunks: {e}")
finally:
    # Close the connection
    connection.close()
    logging.info("Database connection closed.")

<timed exec>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
2025-03-26 09:42:49,711 - INFO - Processing chunk with 10000 rows
2025-03-26 09:42:49,779 - INFO - Memory usage after chunk: 10.99 MB
2025-03-26 09:42:55,283 - INFO - Processing chunk with 10000 rows
2025-03-26 09:42:55,355 - INFO - Memory usage after chunk: 10.98 MB
2025-03-26 09:43:00,929 - INFO - Processing chunk with 10000 rows
2025-03-26 09:43:00,989 - INFO - Memory usage after chunk: 11.06 MB
2025-03-26 09:43:06,581 - INFO - Processing chunk with 10000 rows
2025-03-26 09:43:06,642 - INFO - Memory usage after chunk: 11.00 MB
2025-03-26 09:43:12,971 - INFO - Processing chunk with 10000 rows
2025-03-26 09:43:13,033 - INFO - Memory usage after chunk: 11.13 MB
2025-03-26 09:43:19,350 - INFO - Processing chunk with 10000 rows
2025-03-26 09:43:19,411 - INFO - Memory usage a

CPU times: user 31.6 s, sys: 1.79 s, total: 33.4 s
Wall time: 6min 48s


In [9]:
# Concatenate all chunks into a single DataFrame
df = pd.concat(df_list, ignore_index=True)
# Display a preview
logging.info(f"Final DataFrame shape: {df.shape}")
print(df.head())

2025-03-26 09:58:30,448 - INFO - Final DataFrame shape: (317393, 29)


           merchant                    customer_name  \
0            Exness  CHRISTOPH KWASHIE CASS-GBOLONYO   
1  Nexasoft Limited                  ISSAH  Mohammed   
2            Exness              PRINCE MONPHY ASARE   
3            Exness    DESMOND AKANPAADGI AKANPAADGI   
4  Nexasoft Limited                Prosper  Amekudzi   

              customer_email customer_phone_number payment_reference  \
0    christophcass@yahoo.com         +233207966571         241876174   
1  karamajohnson84@gmail.com         +233261885518  5047755010193684   
2      monphydagr8@gmail.com         +233535710960         241876625   
3    akanpaadgi143@gmail.com         +233546519263         241876640   
4       hamekudzi2@gmail.com         +233505405723    82167010164666   

  transaction_reference  merchant_bears_cost  payment_created_at  \
0   KPY-CA-Xnh5YuApnGzI                    1 2025-01-21 00:00:07   
1   KPY-CA-d53Q7tQwsRhy                    1 2025-01-21 00:02:39   
2   KPY-CA-m6BoggnmrlIe   

In [11]:
df.to_csv('data/fresh_mob_money_data_2.csv', index=False)